In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets('./inputs/mnist')
# Resetting default graph, starting from scratch
tf.reset_default_graph()

epochs = 6000
batch_size = 64
n_noise = 200
learning_rate=0.00015

real_images = tf.placeholder(dtype=tf.float32, shape=[None, 28, 28, 1], name='real_images')
noise = tf.placeholder(dtype=tf.float32, shape=[None, n_noise])

# The keep_prob variable will be used by our dropout layers, which we introduce for more stable learning outcome
keep_prob = tf.placeholder(dtype=tf.float32, name='keep_prob')
is_training = tf.placeholder(dtype=tf.bool, name='is_training')

# Leaky Relu activation
# https://en.wikipedia.org/wiki/Rectifier_%28neural_networks%29#Potential_problems
def lrelu(x):
    return tf.maximum(x, tf.multiply(x, 0.2))

# Binary cross entropy for descriminators
def binary_cross_entropy(x, z):
    eps = 1e-12
    return (-(x * tf.log(z + eps) + (1. - x) * tf.log(1. - z + eps)))

# Code by Parag Mital (github.com/pkmital/CADL)
def montage(images):
    if isinstance(images, list):
        images = np.array(images)
    img_h = images.shape[1]
    img_w = images.shape[2]
    n_plots = int(np.ceil(np.sqrt(images.shape[0])))
    m = np.ones((images.shape[1] * n_plots + n_plots + 1, images.shape[2] * n_plots + n_plots + 1)) * 0.5
    for i in range(n_plots):
        for j in range(n_plots):
            this_filter = i * n_plots + j
            if this_filter < images.shape[0]:
                this_img = images[this_filter]
                m[1 + i + i * img_h:1 + i + (i + 1) * img_h,
                  1 + j + j * img_w:1 + j + (j + 1) * img_w] = this_img
    return m

Extracting ./inputs/mnist\train-images-idx3-ubyte.gz
Extracting ./inputs/mnist\train-labels-idx1-ubyte.gz
Extracting ./inputs/mnist\t10k-images-idx3-ubyte.gz
Extracting ./inputs/mnist\t10k-labels-idx1-ubyte.gz


In [7]:
class DCGAN:
    def __init__(self, img_length, num_colors, d_sizes, g_sizes, keep_prob=keep_prob):
        
        self.img_length = img_length
        self.num_colors = num_colors
        self.latent_dims = g_sizes['z']
        self.momentum = 0.99
        self.keep_prob = keep_prob

        # For mnist, img_length = 28, num_colors = 1
        self.real_images = tf.placeholder(
            dtype=tf.float32,
            shape=[None, img_length, img_length, num_colors],
            name='real_images')
        
        noise = tf.placeholder(
            dtype=tf.float32,
            shape=[None, latent_dims])
        
    def build_discriminator(self, X, reuse=None, keep_prob, d_sizes):
        '''
        d_sizes = {
            'conv_layers': [],
            'dense_layers': []
        }
        '''
        activation = lrelu
        with tf.variable_scope('disc', reuse=reuse):
            # Reshaping input
            x = tf.reshape(X, shape=[-1, self.img_length, self.img_length, 1])
            for kernels, filtersz, stride, dropout, apply_batch_norm in d_sizes['conv_layers']:
                x = tf.layers.conv2d(
                    x,
                    kernel_size=kernels,
                    filters=filtersz,
                    strides=stride,
                    padding='same',
                    activation=activation)
                if dropout:
                    x = tf.layers.dropout(x, self.keep_prob)
            
                if apply_batch_norm:
                    x = tf.contrib.layers.batch_norm(x, is_training=is_training, decay=momentum)
            for units in d_sizes['dense_layers']:
                x = tf.layers.dense(x, units=units, activation=activation)
            x = tf.layers.dense(x, units=1, activation=tf.nn.sigmoid)
            return x
    
    def build_generator(self, Z, reuse=None, keep_prob, g_sizes):
        '''
        # Example
        g_sizes = {
          'd1': 4,
          'd2': 1,
          'dense_layers': [],
          'conv_layers': []
        }
        '''
        activation = lrelu
        with tf.variable_scope('gen', reuse=reuse):
            x = Z
            d1 = d_sizes['d1']
            d2 = d_sizes['d2']
            # Dense layers
            for units, dropout, apply_batch_norm in d_sizes['dense_layers']:
                x = tf.layers.dense(x, units=units, activation=activation)
                if dropout:
                    x = tf.layers.dropout(x, keep_prob)
                if apply_batch_norm:
                    x = tf.contrib.layers.batch_norm(x, is_training=is_training, decay=momentum)
            x = tf.reshape(x, shape=[-1, d1, d1, d2])
            x = tf.image.resize_images(x, size=[7, 7])
            # Conv layers
            for kernels, filtersz, stride, dropout, apply_batch_norm in d_sizes['conv_layers']:
                 x = tf.layers.conv2d_transpose(
                     x,
                     kernel_size=kernels,
                     filters=filtersz,
                     strides=stride,
                     padding='same',
                     activation=activation)
                if dropout:
                    x = tf.layers.dropout(x, self.keep_prob)
                if apply_batch_norm:
                    x = tf.contrib.layers.batch_norm(x, is_training=self.is_training, decay=self.momentum)
            return x

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 82)